In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
from pandas import Series

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [8]:
# Save references to each table
Mmt = Base.classes.measurement
Stn = Base.classes.station

In [9]:
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [10]:
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [11]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [22]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

maxdate = session.query(Mmt.date).order_by(Mmt.date.desc()).first()

AttributeError: type object 'datetime.date' has no attribute 'maxdate'

In [31]:
mindate = dt.date(2017, 8, 23) - dt.timedelta(days=365)

AttributeError: 'str' object has no attribute 'year'

In [ ]:
sel = [Mmt.prcp,Mmt.date]
Precip = session.query(*sel).filter(Mmt.date > mindate).all()

In [ ]:
df = pd.DataFrame(Precip, columns=["Precipitation", "Date"])

In [ ]:
df = df.set_index("Date")
df.head()

In [ ]:
df.plot()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?
station_count =session.query(Mmt.station,func.count(Mmt.station)).group_by(Mmt.station).order_by(func.count(Mmt.station).desc()).all()
len(station_count)

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
station_count

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
sel = [func.min(Mmt.tobs),func.max(Mmt.tobs),func.avg(Mmt.tobs)]
Temps = session.query(*sel).\
    filter(Mmt.date > mindate).\
    filter(Mmt.station == 'USC00519281').all()
Temps



In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
Temp_obs =session.query(Mmt.station,func.count(Mmt.tobs)).group_by(Mmt.station).order_by(func.count(Mmt.tobs).desc()).all()
Temp_obs

In [ ]:
Temps2 = session.query(Mmt.tobs).\
    filter(Mmt.date > mindate).\
    filter(Mmt.station == 'USC00519281').order_by(Mmt.tobs).all()
Temps2

In [ ]:
Temps2lst=[]
Temps2lst=[result[0] for result in Temps2[:-1]]
Temps2lst

In [ ]:
hist_data = pd.Series(Temps2lst)
hist_data.head()

In [ ]:
n_bins = 12
plt.hist(hist_data, bins=n_bins, histtype='bar', label='tobs')
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.legend(prop={'size': 10})
plt.show()

![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
    
def calc_temps(start_date, end_date):

    stdt = func.strftime("%Y-%m-%d",start_date)
    enddt = func.strftime("%Y-%m-%d",end_date)  
       
    return session.query(func.min(Mmt.tobs), func.avg(Mmt.tobs), func.max(Mmt.tobs)).\
        filter(Mmt.date >= stdt).filter(Mmt.date <= enddt).all()


In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
print(calc_temps('2012-02-28', '2012-03-05'))


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
chartdata2=calc_temps('2012-02-28', '2012-03-05')

In [ ]:
fig, ax = plt.subplots()
ax.bar('Temp',chartdata2[0][1],yerr=(chartdata2[0][2]-chartdata2[0][0]))
ax.set_title('Trip Avg Temp')
fig.tight_layout()
plt.show()

In [ ]:
# Calculate the rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
sel2 = [Mmt.station,Stn.name,Stn.latitude,Stn.longitude,Stn.elevation,func.sum(Mmt.prcp)]
Precip = session.query(*sel2).filter(Mmt.station == Stn.station).\
                            filter(Mmt.date > mindate).\
                            group_by(Mmt.station,Stn.latitude,Stn.longitude,Stn.name,Stn.elevation).order_by(func.sum(Mmt.prcp).desc()).all()
df2 = pd.DataFrame(Precip, columns=["Station","Name","Latitude","Longitude","Elevation","Precipitation"])
df2

## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
def daily_normals(date):
     
    sel3 = [func.min(Mmt.tobs), func.avg(Mmt.tobs), func.max(Mmt.tobs)]
    return session.query(*sel3).filter(func.strftime("%m-%d", Mmt.date) == date).distinct()
    

In [ ]:
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`
# Set the start and end date of the trip
# Use the start and end date to create a range of dates
# Stip off the year and save a list of %m-%d strings
# Loop through the list of %m-%d strings and calculate the normals for each date
lstnormals=[]
stdte = func.strftime("%Y-%m-%d",'2012-02-28')
enddte = func.strftime("%Y-%m-%d",'2012-03-05') 
tripdates=session.query(Mmt.date).\
        filter(Mmt.date >= stdte).filter(Mmt.date <= enddte).order_by(Mmt.date.desc()).distinct()
for dt in tripdates:
    appitm = daily_normals(func.strftime("%m-%d",dt[0]))[0]
    lstnormals.append(dt+appitm)
lstnormals  



In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
triptdf = pd.DataFrame(lstnormals, columns=(['Date','Min',"Avg","Max"]))
triptdf = triptdf.set_index('Date')

plt.fill_between(triptdf.index,triptdf['Max'], alpha = .5)
plt.fill_between(triptdf.index,triptdf['Avg'], alpha = .5)
plt.fill_between(triptdf.index,triptdf['Min'], alpha = .5)
plt.title('Weather During Trip')
plt.legend(['Tmax','Tavg','Tmin'],loc = 'lower left')
plt.xticks(rotation=45)
plt.xlabel('Date')
fig.tight_layout()
plt.show()

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
